In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split
import sys
sys.path.append("..")

In [10]:
customers = pd.read_csv('recommend_1.csv') 
transactions = pd.read_csv('trx_data.csv')

In [27]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(3).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0
2,2,111.0,107.0,29.0,11.0,11.0,11.0,33.0,23.0,NaN,NaN


In [23]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [24]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.14 minutes


In [28]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [29]:
data_dummy = create_data_dummy(data)

In [31]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
(df_matrix.shape)

(24429, 300)

In [33]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [38]:
def split_data(data):
    
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [35]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [36]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [39]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

# Baseline Model

In [40]:
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [41]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.065584s

106868 observations to process; with 300 unique items.

In [42]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 421230

recommendations finished on 2000/62483 queries. users per second: 570451

recommendations finished on 3000/62483 queries. users per second: 630120

recommendations finished on 4000/62483 queries. users per second: 619963

recommendations finished on 5000/62483 queries. users per second: 665779

...

recommendations finished on 59000/62483 queries. users per second: 681254

recommendations finished on 60000/62483 queries. users per second: 682858

recommendations finished on 61000/62483 queries. users per second: 666346

recommendations finished on 62000/62483 queries. users per second: 658727

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    248    | 3.0833333333333335 |  1   |
|     0      |     37    | 3.046153846153846  |  2   |
|     0      |     34    | 3.0116731517509727 |  3   |
|     0      |     0     | 2.9950932286555445 |  4   |
|     0      |    132    | 2.9285714285714284 |  5   |
|     0      |     27    | 2.8714285714285714 |  6   |
|     0      |     3     | 2.840085287846482  |  7   |
|     0      |    110    | 2.686046511627907  |  8   |
|     0      |     32    | 2.626943005181347  |  9   |
|     0      |    230    | 2.6259541984732824 |  10  |
|     1      |    248    | 3.0833333333333335 |  1   |
|     1      |     37    | 3.046153846153846  |  2   |
|     1      |     34    | 3.0116731517509727 |  3   |
|     1      |     0     | 2.9950932286555445 |  4   |
|     1      |    132    | 2.9285714285714284 |  5   |
|     1   

In [43]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [44]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [45]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23308 users and 300 items.

Data prepared in: 0.058666s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 454752

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     13    |  1.0  |  1   |
|    1553    |    129    |  1.0  |  2   |
|    1553    |     1     |  1.0  |  3   |
|    1553    |     59    |  1.0  |  4   |
|    1553    |    181    |  1.0  |  5   |
|    1553    |    191    |  1.0  |  6   |
|    1553    |    178    |  1.0  |  7   |
|    1553    |    176    |  1.0  |  8   |
|    1553    |     33    |  1.0  |  9   |
|    1553    |    292    |  1.0  |  10  |
|   20400    |     13    |  1.0  |  1   |
|   20400    |    129    |  1.0  |  2   |
|   20400    |     1     |  1.0  |  3   |
|   20400    |     59    |  1.0  |  4   |
|   20400    |    181    |  1.0  |  5   |
|   20400    |    191    |  1.0  |  6   |
|   20400    |    178    |  1.0  |  7   |
|   20400    |    176    |  1.0  |  8   |
|   20400    |     33    |  1.0  |  9   |
|   20400    |    292    |  1.0  |  10  |
|   19750    |     13    |  1.0  |

In [46]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23306 users and 300 items.

Data prepared in: 0.060031s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 362056

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7443609022556391 |  1   |
|    1553    |    247    | 0.33602150537634395 |  2   |
|    1553    |    230    |  0.3260869565217386 |  3   |
|    1553    |    248    | 0.26420454545454547 |  4   |
|    1553    |    125    |  0.2567164179104473 |  5   |
|    1553    |    204    |  0.2390804597701148 |  6   |
|    1553    |    294    | 0.23650793650793622 |  7   |
|    1553    |    276    | 0.22627737226277372 |  8   |
|    1553    |     74    | 0.22037037037037036 |  9   |
|    1553    |    213    | 0.21973094170403604 |  10  |
|   20400    |    226    |  0.7443609022556391 |  1   |
|   20400    |    247    | 0.33602150537634395 |  2   |
|   20400    |    230    |  0.3260869565217386 |  3   |
|   20400    |    248    | 0.26420454545454547 |  4   |
|   20400    |    125    |  0.2567164179104473 |

In [47]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
248    3.083333
37     3.046154
34     3.011673
0      2.995093
132    2.928571
27     2.871429
3      2.840085
110    2.686047
32     2.626943
230    2.625954
245    2.586957
252    2.574074
10     2.563863
226    2.546099
58     2.472527
173    2.446429
129    2.432749
87     2.413223
82     2.405172
91     2.397541
Name: purchase_count, dtype: float64

# Collaborative Filtering Model

## Cosine

In [48]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.058374s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.01848s

recommendations finished on 1000/1000 queries. users per second: 341064

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11799554824829102  |  1   |
|    1553    |     35    |  0.0776361346244812  |  2   |
|    1553    |     1     | 0.07410693168640137  |  3   |
|    1553    |     5     | 0.06844792366027833  |  4   |
|    1553    |     17    | 0.058206474781036376 |  5   |
|    1553    |     21    | 0.05785325765609741  |  6   |
|    1553    |     33    | 0.05752025842666626  |  7   |
|    1553    |     61    |  0.0553023099899292  |  8   |
|    1553    |     8     | 0.054339706897735596 |  9   |
|    1553    |     15    | 0.044683647155761716 |  10  |
|   20400    |     1     | 0.05152618885040283  |  1   |
|   20400    |     56    | 0.04742097854614258  |  2   |
|   20400    |    246    | 0.04239094257354736  |  3   |
|   20400    |    265    | 0.03743147850036621  |  4   |
|   20400    |     26    | 0.03

In [49]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23308 users and 300 items.

Data prepared in: 0.054743s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.02308s

recommendations finished on 1000/1000 queries. users per second: 326584

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.09817191958427429  |  1   |
|    1553    |     35    | 0.09259313344955444  |  2   |
|    1553    |     1     | 0.07517299056053162  |  3   |
|    1553    |     5     |  0.0675702840089798  |  4   |
|    1553    |     21    | 0.06686508655548096  |  5   |
|    1553    |     33    | 0.06113465130329132  |  6   |
|    1553    |     8     | 0.05626460909843445  |  7   |
|    1553    |     15    | 0.054725825786590576 |  8   |
|    1553    |     38    | 0.05039942264556885  |  9   |
|    1553    |     17    | 0.049554139375686646 |  10  |
|   20400    |     26    | 0.05618000030517578  |  1   |
|   20400    |    215    | 0.050029218196868896 |  2   |
|   20400    |    113    | 0.04947024583816528  |  3   |
|   20400    |     6     | 0.04517233371734619  |  4   |
|   20400    |     1     | 0.04

In [50]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23306 users and 300 items.

Data prepared in: 0.063027s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.021802s

recommendations finished on 1000/1000 queries. users per second: 314861

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     54    |          0.0          |  1   |
|    1553    |    214    |          0.0          |  2   |
|    1553    |    107    |          0.0          |  3   |
|    1553    |     1     |          0.0          |  4   |
|    1553    |    283    |          0.0          |  5   |
|    1553    |    166    |          0.0          |  6   |
|    1553    |    111    |          0.0          |  7   |
|    1553    |    199    |          0.0          |  8   |
|    1553    |     20    |          0.0          |  9   |
|    1553    |    139    |          0.0          |  10  |
|   20400    |     54    |          0.0          |  1   |
|   20400    |    214    |          0.0          |  2   |
|   20400    |    107    |          0.0          |  3   |
|   20400    |     1     |          0.0          |  4   |
|   20400    |

## Pearson

In [51]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.059599s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.025297s

recommendations finished on 1000/1000 queries. users per second: 317158

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.0799344738324486 |  1   |
|    1553    |     37    | 3.046153846153845  |  2   |
|    1553    |     34    | 2.9944378726677217 |  3   |
|    1553    |     0     | 2.993303766089167  |  4   |
|    1553    |    132    | 2.9285714285714293 |  5   |
|    1553    |     27    | 2.871428571428572  |  6   |
|    1553    |     3     | 2.8400852878464824 |  7   |
|    1553    |    110    | 2.6795040435569235 |  8   |
|    1553    |     32    | 2.626943005181348  |  9   |
|    1553    |    230    | 2.621230470770188  |  10  |
|   20400    |    248    | 3.0833333333333335 |  1   |
|   20400    |     37    | 3.046153846153845  |  2   |
|   20400    |     34    | 3.0116731517509736 |  3   |
|   20400    |     0     | 2.9928866647059578 |  4   |
|   20400    |    132    | 2.9155554430825377 |  5   |
|   20400 

In [52]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23308 users and 300 items.

Data prepared in: 0.060012s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.027471s

recommendations finished on 1000/1000 queries. users per second: 398724

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     13    |  0.0  |  1   |
|    1553    |    129    |  0.0  |  2   |
|    1553    |     1     |  0.0  |  3   |
|    1553    |     59    |  0.0  |  4   |
|    1553    |    181    |  0.0  |  5   |
|    1553    |    191    |  0.0  |  6   |
|    1553    |    178    |  0.0  |  7   |
|    1553    |    176    |  0.0  |  8   |
|    1553    |     33    |  0.0  |  9   |
|    1553    |    292    |  0.0  |  10  |
|   20400    |     13    |  0.0  |  1   |
|   20400    |    129    |  0.0  |  2   |
|   20400    |     1     |  0.0  |  3   |
|   20400    |     59    |  0.0  |  4   |
|   20400    |    181    |  0.0  |  5   |
|   20400    |    191    |  0.0  |  6   |
|   20400    |    178    |  0.0  |  7   |
|   20400    |    176    |  0.0  |  8   |
|   20400    |     33    |  0.0  |  9   |
|   20400    |    292    |  0.0  |  10  |
|   19750    |     13    |  0.0  |

In [53]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23306 users and 300 items.

Data prepared in: 0.066079s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.028024s

recommendations finished on 1000/1000 queries. users per second: 304321

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7442429595647897 |  1   |
|    1553    |    247    | 0.33602150537634395 |  2   |
|    1553    |    230    |  0.3257846417634384 |  3   |
|    1553    |    248    |  0.2642045454545455 |  4   |
|    1553    |    125    | 0.25649737084089796 |  5   |
|    1553    |    204    | 0.23908045977011497 |  6   |
|    1553    |    294    | 0.23628374853777506 |  7   |
|    1553    |    276    | 0.22607042493611354 |  8   |
|    1553    |     74    |  0.2201914968313994 |  9   |
|    1553    |    213    |  0.2189305854351531 |  10  |
|   20400    |    226    |  0.744360902255639  |  1   |
|   20400    |    247    | 0.33602150537634395 |  2   |
|   20400    |    230    |  0.3260869565217392 |  3   |
|   20400    |    248    |  0.2642045454545455 |  4   |
|   20400    |    125    | 0.25627456867872755 |

# Model Evaluation

In [54]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [55]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13993 queries. users per second: 160256

recommendations finished on 2000/13993 queries. users per second: 184911

recommendations finished on 3000/13993 queries. users per second: 192703

recommendations finished on 4000/13993 queries. users per second: 206452

...

recommendations finished on 10000/13993 queries. users per second: 217581

recommendations finished on 11000/13993 queries. users per second: 212840

recommendations finished on 12000/13993 queries. users per second: 204193

recommendations finished on 13000/13993 queries. users per second: 205664


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0005717144286428915 | 0.0002413338188368571 |
|   2    |  0.002858572143214467 |  0.003455525948915496 |
|   3    |  0.003406465137330568 |  0.006091367679304493 |
|   4    |  0.007539484027728116 |  0.017561133174296823 |
|   5    |  0.006331737297220055 |  0.01831582148526326  |
|   6    |  0.005693322851902138 |  0.01959939978234833  |
|   7    |  0.006156139294136874 |  0.024242991419436827 |
|   8    |  0.005752876438219103 |  0.025719806924695082 |
|   9    |  0.005502751375687854 |  0.02791554765220159  |
|   10   | 0.0052883584649467755 |  0.029863743972583887 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0004453447726438

Per User RMSE (best)
+------------+------------------------+-----

In [56]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13994 queries. users per second: 175871

recommendations finished on 2000/13994 queries. users per second: 203314

recommendations finished on 3000/13994 queries. users per second: 212736

recommendations finished on 4000/13994 queries. users per second: 228402

...

recommendations finished on 10000/13994 queries. users per second: 266951

recommendations finished on 11000/13994 queries. users per second: 267744

recommendations finished on 12000/13994 queries. users per second: 259291

recommendations finished on 13000/13994 queries. users per second: 240451


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.006502786908675153 | 0.003068265084129026 |
|   2    |  0.005359439759897101 | 0.005738621540751012 |
|   3    |  0.005478538421228156 | 0.008595453781604999 |
|   4    |  0.005198656567100173 | 0.010844603644017662 |
|   5    |  0.005416607117336034 | 0.013678187651676327 |
|   6    |  0.005609546948692288 | 0.01713672770625751  |
|   7    |  0.00550235815349435  | 0.019981782049275178 |
|   8    | 0.0052522509646991304 | 0.02188296314153828  |
|   9    | 0.0051212424372350405 |  0.0239607243589307  |
|   10   |  0.005045019293983142 |  0.0263599952482689  |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------

In [57]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13882 queries. users per second: 166806

recommendations finished on 2000/13882 queries. users per second: 191663

recommendations finished on 3000/13882 queries. users per second: 199124

recommendations finished on 4000/13882 queries. users per second: 214362

...

recommendations finished on 10000/13882 queries. users per second: 208524

recommendations finished on 11000/13882 queries. users per second: 207700

recommendations finished on 12000/13882 queries. users per second: 196027

recommendations finished on 13000/13882 queries. users per second: 197917


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0034577150266532205 | 0.0020368960147090173 |
|   2    |  0.002953464918599609 |  0.003408547782557288 |
|   3    | 0.0027133458195264777 |  0.004920097511222727 |
|   4    | 0.0021970897565192376 |  0.005271871991364847 |
|   5    | 0.0022619219132689813 |  0.006775017551562719 |
|   6    |  0.002173077846611916 | 0.0077109674970213815 |
|   7    |  0.002284561714038735 |  0.009534703238780457 |
|   8    | 0.0022511165538106944 |  0.010781607417538778 |
|   9    |  0.002185083801565576 |  0.011641233792220684 |
|   10   | 0.0023411612159631298 |  0.013406979212338629 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13286398620407439

Per User RMSE (best)
+------------+------------------------+----


# Model Selection


Based on RMSE
    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284

    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097

    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026

In [58]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.056305s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.028445s

recommendations finished on 1000/1000 queries. users per second: 319081

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     1     | 0.10348175764083863  |  1   |
|    1553    |     2     |  0.0934672474861145  |  2   |
|    1553    |     35    | 0.07407552003860474  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.06512556076049805  |  5   |
|    1553    |     15    | 0.06476415395736694  |  6   |
|    1553    |     13    | 0.05799754858016968  |  7   |
|    1553    |     20    | 0.05683724880218506  |  8   |
|    1553    |     29    | 0.05509240627288818  |  9   |
|    1553    |     11    | 0.05467898845672607  |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     15    | 0.047683894634246826 |  4   |
|   20400    |     27    | 0.04

In [59]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,1,0.103482,1
1,1553,2,0.093467,2
2,1553,35,0.074076,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [60]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [66]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [67]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 224921

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|248|125|204|294|276|74|213
11,226|247|230|248|125|204|294|276|74|213
12,226|247|230|248|125|204|294|276|74|213
16,226|247|230|248|125|204|294|276|74|213
21,226|247|230|248|125|204|294|276|74|213


In [68]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [74]:
customer_recomendation(4)

recommendedProducts    226|247|230|248|125|204|294|276|74|213
Name: 4, dtype: object

In [76]:
customer_recomendation(16)

recommendedProducts    226|247|230|248|125|204|294|276|74|213
Name: 16, dtype: object